# 通道套利策略：Donchain Channel

股票市場的行情是隨機而起，一個價格的震盪到底隱含了趨勢或是雜訊？通道模型適度的解決了這個問題。它利用**過去一段時間内的價格訊息，繪製出上下兩條通道線 (上、下界)，借此設定股價的相對高低界限。**
Donchain Channel 流行於上個世紀 70 年代，由著名海龜交易員 Richard Donchain 發明，雖然這個方法在今天已經很難套到巨大的利潤，但是我們依然可以從實作的過程中參考前人的如何把交易系統化的思維方式：

Donchain Channel 的規則如下：

- 每天取**過去 20 日的最大收盤價為上界**
- 每天取**過去 20 日的最小收盤價為下界**
- 利用每天的**上界與下界的平均值當作中界**

![](https://www.dropbox.com/s/8yx9cadj4ou9a4u/donchain_channel.png?dl=1)

Donchain Channel 有兩種策略：

策略 A：
    1. 若收盤價大於過去 20 日收盤價的最大值，代表不正常上漲，所以就放空股票，直到回到中心點平倉
    2. 若收盤價小於過去 20 日收盤價的最小值，代表不正常下跌，所以就做多股票，直到回到中心點平倉

策略 B：
    1. 若收盤價大於過去 20 日收盤價的最大值，代表目前是上漲趨勢，所以就做多股票，直到股價跌破中心點平倉
    2. 若收盤價小於過去 20 日收盤價的最小值，代表不正常下跌趨勢，所以就做空股票，直到股價漲破中心點平倉

***筆者碎碎念：大部分網路上的文件都是以策略 B 爲主，不過筆者自己的經驗是，不同策略適用與不同個股，請先將不同策略針對個股進行回測比較結果**

***備注：非投資建議，本課程提供的資料及交易策略，只可作為參考用途，學員在投資前，務請運用個人獨立思考做出抉擇，如因此招致任何損失，概與本課程無涉。**



In [ ]:
import pandas as pd
# 這邊我們使用臺積電的歷史資料來做回測
df = pd.read_excel("2330_歷史資料.xlsx")
df

# 先來計算上，下，與中軌

Donchain Channel 的上，下，與中軌，分別要取得每一個交易日的前 20 的最大值、最小值、及兩者平均值

幸虧兩者透過 `DataFrame` 計算都十分容易：

```python
# 計算最大值
df["max20d"] = df["收盤價"].rolling(20).max()
df["min20d"] = _____________________________

df["mean20d"] = (df["max20d"] + ___________) / 2
```

In [ ]:
df["max20d"] = df["收盤價"].rolling(20).max()
df

In [ ]:
df["min20d"] = _____________________________

df["mean20d"] = (df["max20d"] + ___________) / 2
df

In [ ]:
# 重新將 DataFrame 的索引值設定成 “日期”
df2 = df.set_index("日期")
df2

# 但是...

別忘了wang現在這樣做回測，`max20d` 與 `min20d` 都是包含每一天當天的價格，所以即使當天價格是20日最高，也永遠不會發生突破上軌的現象，所以我們應該把每一天的 `max20d` 以及 `min20d` 指定給前一天；從 Excel 的角度理解，就是要把 `max20d`、`min20d`、`mean20d` 等欄往上移動一個 row，透過 `DataFrame` 我們可以用 `.shift()` 方法做到

```python
df2["max20d"] = df2["max20d"].shift(1)
df2["min20d"] = df2["min20d"].shift(1)
df2["mean20d"] = df2["mean20d"].shift(1)
```

In [ ]:
df2["max20d"] = df2["max20d"].shift(1)
df2

In [ ]:
df2["max20d"] = df2["max20d"].shift(1)
df2["min20d"] = df2["min20d"].shift(1)
df2["mean20d"] = df2["mean20d"].shift(1)
df2

In [ ]:
# 把 2018 年的資料切片出來
df3 = df2[["收盤價", "max20d", "min20d", "mean20d"]]["2018-01-02":"2018-12-28"]
df3

# 針對 DataFrame 寫 For Loop

由於 Donchain Channel 的回測過程有些複雜，單靠我們在上一堂課學過的 `.apply()` 也很難將邏輯表達清楚

這時我們就需要傳統的 `For Loop` 來迭代 `DataFrame` 的每一個 Row 了，這邊就跟大家介紹 `df.iterrows()`：

```python
for index, row in df3.iterrows():
    print(index)
    print(row)
    print(row["max20d"])
    print("========")
```

In [ ]:
for index, row in df3.iterrows():
    print(index)
    print(row["max20d"])
    print("========")

In [ ]:
import numpy as np

df3["交易訊號"] = ""
df3["損益"] = np.nan
df3

# 實作Donchain Channel 回測（策略A）

In [ ]:
# 目前是否已做多或做空的訊號
hold_flag = False
# 買入股票的訊號
long_flag = False
# 做空股票的訊號
short_flag = False
# 記錄總收益
balance = 0

for index, row in df3.iterrows():
    print(index)
    print(row)
    # 若目前沒有進行交易
    if hold_flag == False:
        # 價格低於下限
        if row["收盤價"] <= row["min20d"]:
            # 顯示做多
            df3.loc[index, "交易訊號"] = "Long"
            # 計算總收益
            balance = balance - (row["收盤價"] * 1000)
            # 花費現金購買股票
            df3.loc[index, "損益"] = balance
            # 開啓交易訊號
            long_flag = True
            hold_flag = True
            # 價格高於上限
        elif row["收盤價"] >= row["max20d"]:
            # 顯示做空
            df3.loc[index, "交易訊號"] = "Short"
            # 做空，現金增加
            balance = balance + (row["收盤價"] * 1000)
            df3.loc[index, "損益"] = balance
            # 開啓交易訊號
            short_flag = True
            hold_flag = True
        else:
            df3.loc[index, "交易訊號"] = "---"
            df3.loc[index, "損益"] = balance
    # 若目前有進行交易
    elif hold_flag == True:
       # 若現在是做多，而且價格大於等於平均值，平倉
       if long_flag == True and row["收盤價"] >= row["mean20d"]:
            df3.loc[index, "交易訊號"] = "Offset"
            # 做多在平倉時，現金增加
            balance = balance + row["收盤價"] * 1000
            df3.loc[index, "損益"] = balance
            hold_flag = False
            long_flag = False
            # 平倉之後，將收益歸零
            balance = 0
       # 若現在是做空，而且價格小於等於平均值，平倉
       elif short_flag == True and row["收盤價"] <= row["mean20d"]:
            df3.loc[index, "交易訊號"] = "Offset"
            # 做空在平倉時，現金減少
            balance = balance - row["收盤價"] * 1000
            df3.loc[index, "損益"] = balance
            # 平倉之後，將收益歸零
            hold_flag = False
            short_flag = False
            balance = 0
       else:
            df3.loc[index, "交易訊號"] = "---"
            df3.loc[index, "損益"] = balance

df3

# 計算之後

我們需要衡量回測的績效

In [ ]:
# 將所有平倉的資料過濾出來
result_df = df3[df3["交易訊號"] == "Offset"]
result_df

# 接下來我們來計算獲勝率


**獲勝率 = 交易有獲利的次數 / 交易總次數**


In [ ]:
# 先過濾出所有賺錢的交易
result_df[result_df["損益"] > 0]

In [ ]:
# 計算這些交易的筆數
result_df[result_df["損益"] > 0].shape

In [ ]:
# 交易總次數
result_df.shape[0]

In [ ]:
# 最後算出我們的獲勝率
winrate_A = result_df[result_df["損益"] > 0].shape[0] / result_df.shape[0]
winrate_A

In [ ]:
# 計算纍積損益
result_df["損益"].cumsum()

In [ ]:
result_df["纍積損益"] = result_df["損益"].cumsum()
result_df

In [ ]:
profit_A = result_df["纍積損益"][-1]
profit_A

In [ ]:
df4 = df2[["收盤價", "max20d", "min20d", "mean20d"]]["2018-01-02":"2018-12-28"]
df4

# 實作策略 B

In [ ]:
# 目前是否已做多或做空的訊號
hold_flag = False
# 買入股票的訊號
long_flag = False
# 做空股票的訊號
short_flag = False
# 記錄收益
balance = 0

# number of shares
shares = 1000

for index, row in df4.iterrows():
    print(index)
    print(row)
    # 若目前沒有進行交易
    if hold_flag == False:
        # 價格低於下限
        if row["收盤價"] <= row["min20d"]:
            # 顯示做空
            df4.loc[index, "交易訊號"] = "Short"
            # 計算總收益
            balance = balance + (row["收盤價"] * shares)
            # 花費現金購買股票
            df4.loc[index, "損益"] = balance
            # 開啓交易訊號
            short_flag = True
            hold_flag = True
            # 價格高於上限
        elif row["收盤價"] >= row["max20d"]:
            # 顯示做多
            df4.loc[index, "交易訊號"] = "Long"
            # 做多，現金減少
            balance = balance - (row["收盤價"] * shares)
            df4.loc[index, "損益"] = balance
            # 開啓交易訊號
            long_flag = True
            hold_flag = True
        else:
            df4.loc[index, "交易訊號"] = "---"
            df4.loc[index, "損益"] = balance
    # 若目前有進行交易
    elif hold_flag == True:
       # 若現在是做多，而且價格小於等於平均值，平倉
       if long_flag == True and row["收盤價"] <= row["mean20d"]:
            df4.loc[index, "交易訊號"] = "Offset"
            # 做多在平倉時，現金增加
            balance = balance + row["收盤價"] * shares
            df4.loc[index, "損益"] = balance
            hold_flag = False
            long_flag = False
            # 平倉之後，將收益歸零
            balance = 0
       # 若現在是做空，而且價格大於等於平均值，平倉
       elif short_flag == True and row["收盤價"] >= row["mean20d"]:
            df4.loc[index, "交易訊號"] = "Offset"
            # 做空在平倉時，現金減少
            balance = balance - row["收盤價"] * shares
            df4.loc[index, "損益"] = balance
            hold_flag = False
            short_flag = False
            # 平倉之後，將收益歸零
            balance = 0
       else:
            df4.loc[index, "交易訊號"] = "---"
            df4.loc[index, "損益"] = balance
    
    print(balance)
    
df4

In [ ]:
# 將所有平倉的資料過濾出來
df4[df4["交易訊號"] == "Offset"]

In [ ]:
# 將所有平倉的資料過濾出來
result_df2 = df4[df4["交易訊號"] == "Offset"]
result_df2

In [ ]:
winrate_B = result_df2[result_df2["損益"] > 0].shape[0] / result_df2.shape[0]
winrate_B

In [ ]:
result_df2["損益"].cumsum()

In [ ]:
result_df2["纍積損益"] = result_df2["損益"].cumsum()
result_df2

In [ ]:
profit_B = result_df2["纍積損益"][-1]
profit_B

In [ ]:
# 將結果輸出至 Excel
import xlwings as xw

wb = xw.Book()

sheet = wb.sheets.add(name="Donchain 策略A")
sheet.range("A1").value = result_df
sheet.range("K1").value = "獲勝率"
sheet.range("L1").value = winrate_A
sheet.range("K2").value = "纍計收益"
sheet.range("L2").value = profit_A

sheet2 = wb.sheets.add(name="Donchain 策略B")
sheet2.range("A1").value = result_df2
sheet2.range("K1").value = "獲勝率"
sheet2.range("L1").value = winrate_B
sheet2.range("K2").value = "纍計收益"
sheet2.range("L2").value = profit_B
wb.save("2330 Donchian Channel 回測")